# SUD tarzan tool: ETL Walkthrough incremental loading

## Outline
 
> This notebook provides a comprehensive overview of ETL process behind the SUD Box Rejects Analysis dashboard. The notebook covers the part of the process related to generation of of the following tables behind the dashboard:
 * SUD_BOX_TOTALS
 * SUD_BOX_REJECT
 * SUD_BOX_CAUSE_REJECTS_DIM
 * SUD_BOX_LINE_RECIPE_DIM




## Questions
1.

## Setup



### Imports

In [1]:

import pandas as pd
import pyodbc as pyodbc

import logging
import queries

import sys
import os
import logging
import sys
import argparse
import pytz
from os.path import join, split 

from datetime import datetime, timedelta
from tomlkit import parse, dumps, loads
from typing import List, Tuple

import historian


from sud_tools import *
from sud_utils import *
from tool_utils import *



### Reading in the configuration file

The configuration file contains all parameters required for the orchestration of the ETL process. This data is stored in toml file, path to which is provided as a parameter of an ETL job. In case if a configuration file is not provided the *ValueError* is raised.

In [2]:
cfg_file_nm = 'etl_config_temp.toml'
cfg_file_path = join(os.getcwd(), cfg_file_nm)
# checking if config file exists
if not os.path.isfile(cfg_file_path):
    raise ValueError(f'No config file was found at: {cfg_file_path}')
else:
    # reading in the config file
    cfg = loads(open(cfg_file_path).read())

# main ETL

## Configuration

### Configuration setting and getting all tag lists

In this section all configuration parameters required for the orchestration of the ETL are getting initialized with the values provided in the configuration file. The ETL process is carried out for each SUD site seperatly, thus following 2 sections outline General and Site specific configuration settings for the ETL.
1. General Configuration
2. Site Specific Configuration

1. General configuration:


General configuration includes parameters related to job (e.g. identifier, name), logs (path to logs), datalab db connection parameters and Power BI refresh parameters as shown in below sections.

### Setting up the connection to DatalabDB

In [3]:
# datalab integration
datalab_cfg = cfg['datalab_db'].copy()

# setting up run configuration with parameters from cfg file
datalab_db_conn = get_mssql_conn_string(**datalab_cfg['connection'])
datalab_db_access_url = r"{}".format(datalab_cfg['access_token_url']['access_token_url'])
datalab_db_access_token = get_azure_sql_db_access_token(datalab_db_access_url)


box_rejects_table_nm = datalab_cfg['output_tables']['box_rejects_table_nm']
box_production_table_nm = datalab_cfg['output_tables']['box_production_table_nm']
box_reject_cause_table_nm = datalab_cfg['output_tables']['box_reject_cause_table_nm']
box_line_recipe_table_nm = datalab_cfg['output_tables']['box_line_recipe_table_nm']

rejects_agg_freq = datalab_cfg['output_tables']['agg_freq']
totals_agg_freq = datalab_cfg['output_tables']['totals_agg_freq']

pbi_refresh_url = cfg['powerbi']['api_url']
pbi_refresh_app = cfg['powerbi']['app_name']

print(f'Datalab connection parameters: {datalab_db_conn}')

Datalab connection parameters: Server=azpg-sqlserver-fhcenganalyticsdatalab.database.windows.net; Database=DatalabDB; Driver=ODBC Driver 17 for SQL Server;


In [4]:
site_cd = 'ami' # change the code to run initial loading for other site
job_id = 10*cfg['job']['id'] + cfg['sites'][site_cd]['job_id_offset']
job_nm = ': '.join((cfg['job']['name'], site_cd))
job_summary_table_nm = cfg['job']['job_summary_table_nm']

print(f'job_id: {job_id}, job_nm: {job_nm}')

job_id: 41, job_nm: Box_tool: ami


### Fetching the tag list information


In [5]:
# reading the tag list file
tags_list_path = cfg['job']['path_to_tags_list']
print(f'Path to tag list table: {tags_list_path}')

# cheking if the logs folder exist
if not os.path.exists(tags_list_path):
    print(f'INFO: Didn not find logs folder at {tags_list_path}.')
    raise ValueError('Was not able to file tags list. Aborting...')
else:
    tags_list_df = pd.read_excel(tags_list_path)
    print(f'Shape of the tags list table: {tags_list_df.shape}')

Path to tag list table: F:\Ecosystem Non-OneDrive\Development Area\MVP048 SUD Box tool\Data\Input\tag_list.xlsx
Shape of the tags list table: (270, 3)


In [6]:
tags_list_df.head()

,TagName,site,enabled
0,LXXX_Cover_General_ExtractedCartons_Total_Coun...,Urlati,1
1,LXXX_Cover_General_ProducedCovers_Total_Counte...,Urlati,1
2,LXXX_Cover_General_RejectedCovers_0_Counter_Ac...,Urlati,1
3,LXXX_Cover_Reshipper_good_inserted_n,Urlati,0
4,LXXX_Cover_Reshipper_bad_rejected_n,Urlati,0


### Site Specific Configuration

Site specific configuration will include the parameters for getting site and line specific information from configuration file. This is shown in below subsections.


In [7]:
site_cfg = cfg['sites'][site_cd].copy()

site_name = site_cfg['site_name']
site_enabled = site_cfg['enabled']


site_history_buffer_days = site_cfg['history_buffer_days']

site_servers = site_cfg['servers'].copy()
site_lines = site_cfg['lines']['lines']
site_lines_agile = site_cfg['lines']['lines_agile']
site_lines_dim = site_cfg['lines']['lines_dim']
line_maping_dict = dict(zip(site_lines, site_lines_dim))

site_historian_tags = site_cfg['tags'].copy()

site_tz = site_servers['timezone']
site_dttm_format = site_servers['dttm_format']
site_historian_source = site_servers['historian']['source']
site_history_depth = site_servers['historian']['history_depth_days']
site_days_to_retake = site_history_depth + site_history_buffer_days



In [8]:
site_lines_dim

['SUAM-31 Packing']

In [9]:
site_lines

['L313']

In [10]:
line_maping_dict

{'L313': 'SUAM-31 Packing'}

In [11]:
print(f'Site name: {site_name}')
print(f'Site enabled: {site_enabled}')
print(f'Site lines: {site_lines}')
print(f'Site lines dimension: {site_lines_dim}')
print(f'Site lines agile: {site_lines_agile}')
print(f'Site historian source: {site_historian_source}')


Site name: Amiens
Site enabled: True
Site lines: ['L313']
Site lines dimension: ['SUAM-31 Packing']
Site lines agile: ['L313']
Site historian source: proficy


### Reading in the SUD Site and Line dimention tables

In [12]:
query_to_execute = "SELECT * FROM SUD_SITES"
sites_dim_df = pd.read_sql(sql=query_to_execute, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD sites dimention table loaded, shape: {sites_dim_df.shape}')

query_to_execute = "SELECT * FROM SUD_LINES"
lines_dim_df = pd.read_sql(sql=query_to_execute, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD lines dimention table loaded, shape: {lines_dim_df.shape}')

query_to_execute = "SELECT * FROM SUD_BOX_CAUSE_REJECTS_DIM"
reject_cause_dim_df = pd.read_sql(sql=query_to_execute, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD reject cause dimention table loaded, shape: {reject_cause_dim_df.shape}')

query_to_execute = "SELECT * FROM SUD_BOX_LINE_RECIPE_DIM"
line_recipe_dim_df = pd.read_sql(sql=query_to_execute, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD line recipe dimention table loaded, shape: {line_recipe_dim_df.shape}')



SUD sites dimention table loaded, shape: (5, 10)
SUD lines dimention table loaded, shape: (72, 13)
SUD reject cause dimention table loaded, shape: (99, 7)
SUD line recipe dimention table loaded, shape: (20, 5)


### Getting list of all tags and extracting data for all tags

In [13]:
select_mask = (tags_list_df['site'] == site_name) & (tags_list_df['enabled'] == 1)
site_tags = tags_list_df.loc[select_mask, 'TagName'].unique()
site_tags = ['_'.join(t.split('_')[1:]) for t in site_tags]
site_tags[:5]

['Cover_General_ExtractedCartons_Total_Counter_Actual_n',
 'Cover_General_ProducedCovers_Total_Counter_Actual_n',
 'Cover_General_RejectedCovers_0_Counter_Actual_n',
 'Cover_TransportBelt_CheckExternal_0_0_Rejected_n',
 'Cover_TransportBelt_CheckInternal_0_0_Rejected_n']

In [14]:
print(len(site_tags))

119


In [15]:
all_tag_list = get_tags_list(lines=site_lines, sensors=site_tags, sep='_', 
                             topic=site_servers['historian']['proficy']['topic'])
print(f'# Tags to be extracted: {len(all_tag_list)}')


# Tags to be extracted: 119


## ETL SUD_BOX_REJECTS table
**TODO:**
* Add incremantal loading of reject cause dimention table
* Add incremental loading of the line recipe dimention table

### Generating List  of Tags to Extract

In [16]:
#reject_tags = reject_cause_df.loc[:, 'tag_nm'].unique()
reject_tags = reject_cause_dim_df.loc[:, 'tag_nm'].unique().tolist()
reject_tags[:5]

['Cover_TransportBelt_CheckExternal_0_0_Rejected_n',
 'Cover_TransportBelt_CheckInternal_0_0_Rejected_n',
 'Cover_TransportBelt_RobotTrack_Skipped_0_Rejected_n',
 'Cover_Forming_PatchErectionCheck_Bad_Counter_Rejected_n',
 'Cover_Forming_PatchErectionCheck_BadAngle_Counter_Rejected_n']

In [17]:
print(len(reject_tags))

99


In [18]:
#reject cause tags to extract data
reject_cause_tags = get_tags_list(lines=site_lines, sensors=reject_tags,sep='_', topic=site_servers['historian']['proficy']['topic'])
reject_cause_tags[:5]

('AMI-SUDHIS-01.L313_Cover_TransportBelt_CheckExternal_0_0_Rejected_n',
 'AMI-SUDHIS-01.L313_Cover_TransportBelt_CheckInternal_0_0_Rejected_n',
 'AMI-SUDHIS-01.L313_Cover_TransportBelt_RobotTrack_Skipped_0_Rejected_n',
 'AMI-SUDHIS-01.L313_Cover_Forming_PatchErectionCheck_Bad_Counter_Rejected_n',
 'AMI-SUDHIS-01.L313_Cover_Forming_PatchErectionCheck_BadAngle_Counter_Rejected_n')

In [19]:
# Base_Machine_RejectedBases_Station3_Counter to be used in fix for "Other rejects" tag
base_station3_reject_tag = [t for t in all_tag_list if '_'.join(t.split('_')[1:]) in site_historian_tags['tag_station']]
base_station3_reject_tag

['AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station3_Counter_Actual_n']

### Extracting data for Rejects per cause

In [20]:
historian.use_context('REST',
            client_id='historian_public_rest_api',
            client_password='publicapisecret',
            app_id='sudanalytics.im',
            app_password='phoenix2021SUD', # change to environmental variable
            port=site_servers['historian']['proficy']['port'],
            verify_certificate=True
        )

#### Define extract window

In [21]:
# define extract window
last_run_df = get_last_run_df(table_nm=box_rejects_table_nm,
                        db_conn=get_db_connection(datalab_db_conn, datalab_db_access_token),
                        job_nm=job_nm)

if not last_run_df.empty:
    last_available_dttm = last_run_df['last_dttm_available'][0]
    last_available_id = last_run_df['last_id_available'][0]
else:
    last_available_dttm = None
    last_available_id = None

start_dttm = datetime.now() if last_available_dttm is None else last_available_dttm
start_dttm = start_dttm - timedelta(days=site_days_to_retake)
start_time = start_dttm
end_time = datetime.now()
print(f'Extracting data between: {start_time} and {end_time}')

        

Extracting data between: 2023-04-07 05:30:00 and 2023-04-17 08:45:25.367050


## T2 historian tags

In [22]:

all_ami_T2_tags = historian.get_tag_names(site_servers['historian']['proficy']['server_name'])
all_ami_T2_tags = list(all_ami_T2_tags.keys())
all_ami_T2_tags_df = pd.DataFrame({'tag_nm':all_ami_T2_tags})

In [ ]:
absent_tags = [t for t in all_tag_list if t not in all_ami_T2_tags]
print(len(absent_tags), len(all_tag_list))
absent_tags



In [26]:
extracts = []
for line in site_lines:
    logging.info(f'Extracting cause rejects counters for line {line}')
    # extracting checkweigher tag
    #checkweigher_tags_to_extract = [t for t in all_tag_list if ('BoxCheckweigher' in t or '')]
    checkweigher_tags_to_extract = [t for t in all_tag_list if ('BoxCheckweigher' in t or 'PR_Data_State' in t) and line in t ]
    #checkweigher_tags_to_extract = checkweigher_tags_to_extract + ['URL-PACK.L13_Base_General_PR_Data_State_Actual_n']


    checkweigher_extract_df = historian.get_tag_values(
                site_servers['historian']['proficy']['server_name'],
                start_time=start_time,
                end_time=end_time,
                filter_name=checkweigher_tags_to_extract
                )
    logging.info(f'Shape of data extract: {checkweigher_extract_df.shape}')
    checkweigher_extract_df = checkweigher_extract_df.reset_index()
    checkweigher_extract_df = checkweigher_extract_df.assign(Value = pd.to_numeric(checkweigher_extract_df['Value']))

    checkweigher_extract_df = checkweigher_extract_df.sort_values(['Value','Timestamp'], ascending = True)

    mask_status = checkweigher_extract_df['Tag'].str.contains('Base_General_PR_Data_State_Actual_n')
    checkweigher_extract_df = checkweigher_extract_df.assign(status = np.where(mask_status, checkweigher_extract_df['Value'], np.nan))

    checkweigher_extract_df = checkweigher_extract_df.sort_values(by=['Timestamp'])
    checkweigher_extract_df = checkweigher_extract_df.assign(status = checkweigher_extract_df['status'].ffill())
    checkweigher_extract_df = checkweigher_extract_df.loc[~mask_status]
    group_by=['Tag']
    checkweigher_extract_df = checkweigher_extract_df.assign(rejects_qty = checkweigher_extract_df.groupby(group_by)['Value'].diff().values)
    #mask_sm =(checkweigher_extract_df['rejects_qty'] >=10) & (checkweigher_extract_df['status'] == 1) 
    mask_sm =(checkweigher_extract_df['rejects_qty'] >=10) & ((checkweigher_extract_df['status'] == 1) | (checkweigher_extract_df['status'] == 2) | (checkweigher_extract_df['status'] == 3) | (checkweigher_extract_df['status'] == 4) ) 

    checkweigher_extract_df = checkweigher_extract_df.loc[~mask_sm]
    logging.info(f'Shape of after spikes: {checkweigher_extract_df.shape}')
    checkweigher_extract_df = checkweigher_extract_df[['Tag','Timestamp','Value','Quality']]
    reject_cause_tags = [t for t in reject_cause_tags if t not in checkweigher_tags_to_extract]

    tags_to_extr = [t for t in list(reject_cause_tags) + base_station3_reject_tag if line in t and t not in checkweigher_tags_to_extract]

    historian_extract_df = historian.get_tag_values(
                        site_servers['historian']['proficy']['server_name'],
                        start_time=start_time,
                        end_time=end_time,
                        filter_name=tuple(tags_to_extr))
    logging.info(f'Shape of data extract: {historian_extract_df.shape}')
    #extracts.append(historian_extract_df.reset_index())
    historian_extract_df = historian_extract_df.reset_index()
    extract_df = pd.concat([historian_extract_df, checkweigher_extract_df], ignore_index=True, sort=False)
    extracts.append(extract_df)

#reject_historian_extract_df = pd.concat(extracts, ignore_index=True, sort=False).set_index(['Tag', 'Timestamp'])
reject_historian_extract_df = pd.concat(extracts, ignore_index=True, sort=False).set_index(['Tag', 'Timestamp'])


print(f'Shape of data extract: {reject_historian_extract_df.shape}')


Shape of data extract: (20760, 2)


In [27]:
checkweigher_extract_df

,Tag,Timestamp,Value,Quality
12693,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-11 12:01:55.764000+00:00,17,3
12692,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-11 12:05:30.769000+00:00,24,3
12691,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-11 12:05:32.785000+00:00,32,3
12690,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-11 12:05:42.769000+00:00,33,3
12689,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-11 12:35:12.818000+00:00,35,3
...,...,...,...,...
219,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-17 08:40:44.843000+00:00,10169,3
218,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-17 08:42:47.829000+00:00,10170,3
217,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Zon...,2023-04-17 08:42:48.829000+00:00,10177,3
13321,AMI-SUDHIS-01.L313_Upack_BoxCheckweigher_0_Ext...,2023-04-17 08:42:48.829000+00:00,16,3


#### Extract Line Recipe

In [28]:
line_recipe_tags = [t for t in all_tag_list if '_'.join(t.split('_')[1:]) in site_historian_tags['tags_index']]
line_recipe_tags

['AMI-SUDHIS-01.L313_Upack_LineRecipe']

In [29]:
historian_line_recipe_extract_df = historian.get_tag_values(
                    site_servers['historian']['proficy']['server_name'],
                    start_time=start_time - timedelta(days=20),
                    end_time=end_time,
                    filter_name=line_recipe_tags)
print(f'Shape of data extract: {historian_line_recipe_extract_df.shape}')
                

Shape of data extract: (13, 2)


In [30]:
line_recipe_status_df = put_to_wonderware_format(historian_line_recipe_extract_df, inplace=False)
line_recipe_status_df = line_recipe_status_df.assign(DateTime = line_recipe_status_df['DateTime'].dt.tz_convert(site_tz).dt.tz_localize(None),
                                                     Value = pd.to_numeric(line_recipe_status_df['Value']),
                                                     Quality = pd.to_numeric(line_recipe_status_df['Quality']))
line_recipe_status_df = add_site_line_tag(line_recipe_status_df, site=site_name)

keep_mask = line_recipe_status_df['Quality'] == 3
line_recipe_status_df = line_recipe_status_df.loc[keep_mask].drop(columns=['Quality', 'tag'])
print(line_recipe_status_df.shape)
line_recipe_status_df.head()

TRANSFORM: Shape after addition of columns: (13, 6)
(13, 4)


,DateTime,Value,line,site
12,2023-04-06 18:07:28.513,21,L313,Amiens
11,2023-04-07 09:00:24.457,51,L313,Amiens
10,2023-04-12 08:26:28.074,57,L313,Amiens
9,2023-04-12 11:19:58.243,42,L313,Amiens
8,2023-04-13 03:20:37.272,51,L313,Amiens


#### Extract Agile flag

In [31]:
agile_tags = [t for t in all_tag_list if '_'.join(t.split('.')[1].split('_')[1:]) in site_historian_tags['tags_agile']]
agile_tags

['AMI-SUDHIS-01.L313_Upack_Primary_General_Speed_AgileControl_State_ON']

In [32]:
agile_extract_df = historian.get_tag_values(site_servers['historian']['proficy']['server_name'], 
                                            start_time=start_time - timedelta(days=10),
                                            end_time=end_time,
                                            filter_name=agile_tags)
print(f'Shape of agile flag extract: {agile_extract_df.shape}')

Shape of agile flag extract: (0, 2)


In [33]:
if agile_extract_df.shape[0] > 0:
    agile_data_available = True

    agile_status_df = put_to_wonderware_format(agile_extract_df, inplace=False)
    agile_status_df = agile_status_df.assign(DateTime = agile_status_df['DateTime'].dt.tz_convert(site_tz).dt.tz_localize(None),
                                            Value = pd.to_numeric(agile_status_df['Value']),
                                            Quality = pd.to_numeric(agile_status_df['Quality']))
    agile_status_df = add_site_line_tag(agile_status_df, site=site_name)
    
    keep_mask = agile_status_df['Quality'] == 3
    agile_status_df = agile_status_df.loc[keep_mask].drop(columns=['Quality', 'tag'])
else:
    agile_data_available = False 

#### Extract Line State

In [34]:
extract_dttm = datetime.strftime(start_time.date() - timedelta(days=5), '%Y%m%d %H:%M:%S')
print(f'Extracting data starting from: {extract_dttm}')
line_query = f"('{site_lines_dim[0]}')" if len(site_lines) == 1 else tuple(site_lines_dim)
query_to_execute = queries.EXTRACT_LINE_STATE.format(extract_dttm, line_query )
print(f'Extracting data with: {query_to_execute}')

Extracting data starting from: 20230402 00:00:00
Extracting data with: 
    SELECT ls.line_state_id
        ,ls.start_time
        ,ls.end_time
        ,l.line
        ,s.site
    FROM SUD_LINE_STATE as ls
    INNER JOIN SUD_LINES as l 
        ON ls.line_id = l.line_id 
    INNER JOIN SUD_SITES as s
        ON ls.site_id = s.site_id
    WHERE ls.start_time >= '20230402 00:00:00'
    AND l.line in ('SUAM-31 Packing')



In [35]:
site_lines_dim

['SUAM-31 Packing']

In [36]:
line_state_map_df = pd.read_sql(sql=query_to_execute, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'Line state was extracted, shape: {line_state_map_df.shape}')


Line state was extracted, shape: (3674, 5)


In [37]:
index_cols = ['line_state_id', 'line', 'site']
line_state_map_df = (line_state_map_df
                                    .set_index(index_cols)
                                    .stack()
                                    .to_frame('DateTime')
                                    .reset_index()
                                    .drop(columns=[f'level_{len(index_cols)}'])
                                    )
        
line_state_map_df.head()            

,line_state_id,line,site,DateTime
0,2099141/0,SUAM-31 Packing,Amiens,2023-04-02 06:00:00
1,2099141/0,SUAM-31 Packing,Amiens,2023-04-02 14:00:00
2,2099159/0,SUAM-31 Packing,Amiens,2023-04-02 14:00:00
3,2099159/0,SUAM-31 Packing,Amiens,2023-04-02 22:00:00
4,2099192/0,SUAM-31 Packing,Amiens,2023-04-02 22:00:00


### Transformations for SUD_BOX_REJECTS

In [38]:
# process historian extract 
box_rejects_df = transform_counters_extract(reject_historian_extract_df, site_name=site_name, local_tz=site_tz)
print(box_rejects_df.shape)
box_rejects_df.head()

TRANSFORM: Shape of original extracted data: (20760, 3)
TRANSFORM: Step 1 - Removing cases where the counter values were not logged (NaNs) or negative
TRANSFORM: # removed records 0
TRANSFORM: Shape after step: (20760, 3)
TRANSFORM: Step 2 - Removing cases where the counter was not changing
TRANSFORM: # removed records 0
TRANSFORM: Shape after step: (20760, 5)
TRANSFORM: Step 3 - Removing cases where the counter reports zero
TRANSFORM: # removed records 651
TRANSFORM: Shape after step: (20109, 5)
TRANSFORM: Shape after addition of columns: (20109, 5)
(20109, 5)


,DateTime,rejects_qty,line,tag,site
5572,2023-04-11 14:58:33.800,1.0,L313,Base_Checkweigher_Base _2Dcheck_TimeOut_Reject...,Amiens
5571,2023-04-11 14:58:36.800,1.0,L313,Base_Checkweigher_Base _2Dcheck_TimeOut_Reject...,Amiens
5569,2023-04-11 22:03:49.259,1.0,L313,Base_Checkweigher_Base _2Dcheck_TimeOut_Reject...,Amiens
5568,2023-04-12 06:15:50.798,1.0,L313,Base_Checkweigher_Base _2Dcheck_TimeOut_Reject...,Amiens
5566,2023-04-12 14:13:40.324,1.0,L313,Base_Checkweigher_Base _2Dcheck_TimeOut_Reject...,Amiens


In [39]:
tag_ = box_rejects_df.tag.unique().tolist()
tag_

['Base_Checkweigher_Base _2Dcheck_TimeOut_Rejected_n',
 'Base_Checkweigher_Base_2Dcheck_NotRead_Rejected_n',
 'Base_Checkweigher_Cover_2Dcheck_NotRead_Rejected_n',
 'Base_Checkweigher_Cover_2Dcheck_TimeOut_Rejected_n',
 'Base_Checkweigher_Tamper_2Dcheck_MisMatch_Rejected_n',
 'Base_Checkweigher_Tamper_2Dcheck_NotRead_Rejected_n',
 'Base_Checkweigher_Tamper_2Dcheck_TimeOut_Rejected_n',
 'Base_Checkweigher_Washcount_Check_NotPresent_Rejected_n',
 'Base_Checkweigher_Washcount_Check_WrongPosition_Rejected_n',
 'Base_Filling_Ishida_Channel1_CycleNotConfirmed_Rejected_n',
 'Base_Filling_Ishida_Channel1_WrongDoseDumped_Rejected_n',
 'Base_Filling_Ishida_Channel2_CycleNotConfirmed_Rejected_n',
 'Base_Filling_Ishida_Channel2_WrongDoseDumped_Rejected_n',
 'Base_Filling_Ishida_Channel3_CycleNotConfirmed_Rejected_n',
 'Base_Filling_Ishida_Channel3_WrongDoseDumped_Rejected_n',
 'Base_Filling_Ishida_Channel4_CycleNotConfirmed_Rejected_n',
 'Base_Filling_Ishida_Channel4_WrongDoseDumped_Rejected_n',
 

#### Mapping to line recipe

In [40]:
## add line recipe
box_rejects_df = add_dim_key_time(fact_df=box_rejects_df, 
                                  dim_df=line_recipe_status_df.rename(columns={'Value':'recipe_id'}), 
                                  dim_id='recipe_id', fact_dttm='DateTime', dim_dttm='DateTime', 
                                  group_by=['site', 'line'])
print(box_rejects_df.shape)
box_rejects_df.head()

(20109, 6)


,DateTime,rejects_qty,line,tag,site,recipe_id
4445,2023-04-07 07:30:00.236,2184.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0
4446,2023-04-07 07:30:03.236,2.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0
4447,2023-04-07 07:30:04.236,1.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0
4448,2023-04-07 07:30:08.236,2.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0
4449,2023-04-07 07:31:06.233,1.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0


#### Mapping to Agile flag

In [41]:
if agile_data_available:
    # add agile flag
    box_rejects_df = add_dim_key_time(fact_df=box_rejects_df, 
                                      dim_df=agile_status_df.rename(columns={'Value':'agile_flag'}), 
                                      dim_id='agile_flag', fact_dttm='DateTime', dim_dttm='DateTime', 
                                      group_by=['site', 'line'])
    box_rejects_df = box_rejects_df.assign(agile_flag = box_rejects_df['agile_flag'].fillna(0))
else:
    box_rejects_df = box_rejects_df.assign(agile_flag = 0)

print(box_rejects_df.shape)
box_rejects_df.head()

(20109, 7)


,DateTime,rejects_qty,line,tag,site,recipe_id,agile_flag
4445,2023-04-07 07:30:00.236,2184.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0
4446,2023-04-07 07:30:03.236,2.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0
4447,2023-04-07 07:30:04.236,1.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0
4448,2023-04-07 07:30:08.236,2.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0
4449,2023-04-07 07:31:06.233,1.0,L313,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0


#### Mapping to Line State

In [42]:
 ## add line state
box_rejects_df = add_dim_key_time(fact_df=box_rejects_df.assign(line = box_rejects_df['line'].map(line_maping_dict)), 
                                  dim_df=line_state_map_df, 
                                  dim_id='line_state_id', 
                                  fact_dttm='DateTime', dim_dttm='DateTime', 
                                  group_by=['site', 'line'])
print(box_rejects_df.shape)
box_rejects_df.head()

(20109, 8)


,DateTime,rejects_qty,line,tag,site,recipe_id,agile_flag,line_state_id
0,2023-04-07 07:30:00.236,2184.0,SUAM-31 Packing,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0.0,2099894/19
1,2023-04-07 07:30:03.236,2.0,SUAM-31 Packing,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0.0,2099894/19
2,2023-04-07 07:30:04.236,1.0,SUAM-31 Packing,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0.0,2099894/19
3,2023-04-07 07:30:08.236,2.0,SUAM-31 Packing,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0.0,2099894/19
4,2023-04-07 07:31:06.233,1.0,SUAM-31 Packing,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,Amiens,21.0,0.0,2099894/19


#### Aggregating data

In [43]:
## aggregating
box_rejects_df = box_rejects_df.assign(DateTimeMin = box_rejects_df['DateTime'].dt.floor(rejects_agg_freq))

groupby_cols = ['site', 'line', 'DateTimeMin', 'tag', 
                'agile_flag', 'recipe_id', 'line_state_id']
agg_dict = {'rejects_qty':'sum', 'DateTime':'min'}

box_rejects_agg_df = box_rejects_df.groupby(groupby_cols).agg(agg_dict).reset_index()
print(box_rejects_agg_df.shape)
box_rejects_agg_df.head()


(4546, 9)


,site,line,DateTimeMin,tag,agile_flag,recipe_id,line_state_id,rejects_qty,DateTime
0,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadAngle_Coun...,0.0,21.0,2099894/31,25.0,2023-04-07 07:59:27.262
1,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadAngle_Coun...,0.0,21.0,2099894/32,2.0,2023-04-07 07:59:51.263
2,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,0.0,21.0,2099894/19,2195.0,2023-04-07 07:30:00.236
3,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,0.0,21.0,2099894/20,12.0,2023-04-07 07:40:09.249
4,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,0.0,21.0,2099894/21,35.0,2023-04-07 07:40:30.247


####  Fixing "Other Rejects" Counter

In [44]:
print(box_rejects_agg_df.shape)
box_rejects_fix_df = get_other_rejects_tag_fixed(box_rejects_agg_df)
print(box_rejects_fix_df.shape)

(4546, 9)
(166, 8)


In [45]:
site_historian_tags['tag_station']

['Base_Machine_RejectedBases_Station3_Counter_Actual_n']

In [46]:
print(box_rejects_agg_df.shape)
box_rejects_agg_df = box_rejects_agg_df.loc[~box_rejects_agg_df['tag'].isin(site_historian_tags['tag_station'])]
print(box_rejects_agg_df.shape)
box_rejects_agg_df = box_rejects_agg_df.merge(box_rejects_fix_df, how='left', on=groupby_cols)
print(box_rejects_agg_df.shape)

(4546, 9)
(4410, 9)
(4410, 10)


In [47]:
if site_cd == 'url':
    adjust_mask = ((box_rejects_agg_df['tag'] == 'Upack_BoxCheckweigher_0_Extra_Rejected_n') & 
                    (box_rejects_agg_df['DateTimeMin'] >= '2022-07-14') # date when the tags were fixed in historian
                    )
else:
    adjust_mask = box_rejects_agg_df['tag'] == 'Upack_BoxCheckweigher_0_Extra_Rejected_n'

box_rejects_agg_df = (box_rejects_agg_df
                        .assign(rejects_qty = np.where(adjust_mask, box_rejects_agg_df['new_rejects_qty'], box_rejects_agg_df['rejects_qty']))
                        .drop(columns=['new_rejects_qty'])
            )

print(box_rejects_agg_df.shape)
box_rejects_agg_df.head()

(4410, 9)


,site,line,DateTimeMin,tag,agile_flag,recipe_id,line_state_id,rejects_qty,DateTime
0,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadAngle_Coun...,0.0,21.0,2099894/31,25.0,2023-04-07 07:59:27.262
1,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadAngle_Coun...,0.0,21.0,2099894/32,2.0,2023-04-07 07:59:51.263
2,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,0.0,21.0,2099894/19,2195.0,2023-04-07 07:30:00.236
3,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,0.0,21.0,2099894/20,12.0,2023-04-07 07:40:09.249
4,Amiens,SUAM-31 Packing,2023-04-07 07:30:00,Cover_Forming_PatchErectionCheck_BadHeight_Cou...,0.0,21.0,2099894/21,35.0,2023-04-07 07:40:30.247


In [48]:

# renaming columns
rename_dict = {'DateTimeMin':'datetime', 
                'DateTime':'start_time'}
box_rejects_agg_df = (box_rejects_agg_df
                                        .merge(lines_dim_df[['line_id', 'line']], on='line', how='inner')
                                        .merge(sites_dim_df[['site_id', 'site']], on='site', how='inner')
                                        .merge(reject_cause_dim_df[['reject_cause_id', 'tag_nm']], left_on='tag', right_on='tag_nm', how='inner')
                                        .drop(columns=['site', 'line', 'tag', 'tag_nm'])
                                        .rename(columns=rename_dict)
                                    )
print(f'Shape of resulting box rejects data: {box_rejects_agg_df.shape}')
                

Shape of resulting box rejects data: (3453, 9)


In [49]:
box_rejects_agg_df[['line_state_id','recipe_id','line_id','site_id','reject_cause_id']].isnull().any()

line_state_id      False
recipe_id          False
line_id            False
site_id            False
reject_cause_id    False
dtype: bool

## ETL for generating SUD_BOX_TOTALS table

#### Generating list of tags to extract

In [50]:
tags_total = [t for t in site_historian_tags['tags_total']]
tags_total

['Cover_General_ExtractedCartons_Total_Counter_Actual_n',
 'Cover_General_ProducedCovers_Total_Counter_Actual_n',
 'Base_Machine_ExtractedCartons_0_Counter_Actual_n',
 'Base_Machine_ProducedBases_0_Counter_Actual_n',
 'Base_Machine_RejectedBases_Station1_Counter_Actual_n',
 'Base_Machine_RejectedBases_Station2_Counter_Actual_n',
 'Base_Machine_RejectedBases_Station3_Counter_Actual_n',
 'Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n',
 'Cover_General_RejectedCovers_0_Counter_Actual_n',
 'Upack_Produced_Case_At_Spiral_n']

In [51]:
reject_total_tags = get_tags_list(lines=site_lines, sensors=tags_total, sep='_', topic=site_servers['historian']['proficy']['topic'])
reject_total_tags

('AMI-SUDHIS-01.L313_Cover_General_ExtractedCartons_Total_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Cover_General_ProducedCovers_Total_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_ExtractedCartons_0_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_ProducedBases_0_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station1_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station2_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station3_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n',
 'AMI-SUDHIS-01.L313_Cover_General_RejectedCovers_0_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Upack_Produced_Case_At_Spiral_n')

### Extracting data for SUD_BOX_TOTALS

#### Defining Extract window

In [52]:
# define extract window
last_run_df = get_last_run_df(table_nm=box_production_table_nm,
                        db_conn=get_db_connection(datalab_db_conn, datalab_db_access_token),
                        job_nm=job_nm)

if not last_run_df.empty:
    last_available_dttm = last_run_df['last_dttm_available'][0]
    last_available_id = last_run_df['last_id_available'][0]
else:
    last_available_dttm = None
    last_available_id = None

start_dttm = datetime.now() if last_available_dttm is None else last_available_dttm
start_dttm = start_dttm - timedelta(days=site_days_to_retake)
start_time = start_dttm
end_time = datetime.now()
print(f'Extracting data between: {start_time} and {end_time}')

        

Extracting data between: 2023-04-07 05:30:00 and 2023-04-17 08:49:12.930853


In [53]:
# start_time = datetime(2022, 12, 29)
# end_time = datetime.now()
# print(f'Extracting data between: {start_time} and {end_time}')


#### Extracting Produced/Extracted/Total Rejected Counters

In [54]:
reject_total_tags




('AMI-SUDHIS-01.L313_Cover_General_ExtractedCartons_Total_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Cover_General_ProducedCovers_Total_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_ExtractedCartons_0_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_ProducedBases_0_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station1_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station2_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Station3_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n',
 'AMI-SUDHIS-01.L313_Cover_General_RejectedCovers_0_Counter_Actual_n',
 'AMI-SUDHIS-01.L313_Upack_Produced_Case_At_Spiral_n')

In [55]:
total_historian_extract_df = historian.get_tag_values(
                    site_servers['historian']['proficy']['server_name'],
                    start_time=start_time,
                    end_time=end_time,
                    filter_name=reject_total_tags)
print(f'Shape of data extract: {total_historian_extract_df.shape}')

Shape of data extract: (108893, 2)


In [56]:
## for L27

In [57]:
#.str.split('.', n=1, expand=True)[1] 'L343' in t
# tags_to_extract_total_L27 = [t for t in reject_total_tags if 'L273' in t]
# tags_to_extract_total_L27

In [58]:
# print(end_time)

In [59]:
# total_historian_extract_df_L27_first_chunck = historian.get_tag_values(
#                     site_servers['historian']['proficy']['server_name'],
#                     start_time=start_time,
#                     end_time=end_time,
#                     filter_name=tags_to_extract_total_L27)
# print(f'Shape of data extract: {total_historian_extract_df_L27_second_chunck.shape}')

### Transformations for SUD_BOX_TOTALS

In [60]:
# process historian extract 
box_totals_df = transform_counters_extract(total_historian_extract_df, site_name=site_name, local_tz=site_tz)
box_totals_df

TRANSFORM: Shape of original extracted data: (108893, 3)
TRANSFORM: Step 1 - Removing cases where the counter values were not logged (NaNs) or negative
TRANSFORM: # removed records 0
TRANSFORM: Shape after step: (108893, 3)
TRANSFORM: Step 2 - Removing cases where the counter was not changing
TRANSFORM: # removed records 0
TRANSFORM: Shape after step: (108893, 5)
TRANSFORM: Step 3 - Removing cases where the counter reports zero
TRANSFORM: # removed records 73
TRANSFORM: Shape after step: (108820, 5)
TRANSFORM: Shape after addition of columns: (108820, 5)


,DateTime,rejects_qty,line,tag,site
64113,2023-04-14 11:55:07.274,9610.0,L313,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Amiens
64112,2023-04-14 11:55:08.274,2.0,L313,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Amiens
64111,2023-04-14 11:55:09.274,2.0,L313,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Amiens
64110,2023-04-14 11:55:10.274,3.0,L313,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Amiens
64109,2023-04-14 11:55:11.274,2.0,L313,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Amiens
...,...,...,...,...,...
92261,2023-04-17 10:37:57.764,1.0,L313,Upack_Produced_Case_At_Spiral_n,Amiens
92260,2023-04-17 10:38:10.811,1.0,L313,Upack_Produced_Case_At_Spiral_n,Amiens
92259,2023-04-17 10:38:19.825,1.0,L313,Upack_Produced_Case_At_Spiral_n,Amiens
92258,2023-04-17 10:44:19.873,1.0,L313,Upack_Produced_Case_At_Spiral_n,Amiens


In [61]:
box_totals_df.tag.unique()

array(['Base_Machine_ExtractedCartons_0_Counter_Actual_n',
       'Base_Machine_ProducedBases_0_Counter_Actual_n',
       'Base_Machine_RejectedBases_Station1_Counter_Actual_n',
       'Base_Machine_RejectedBases_Station2_Counter_Actual_n',
       'Base_Machine_RejectedBases_Station3_Counter_Actual_n',
       'Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n',
       'Cover_General_ExtractedCartons_Total_Counter_Actual_n',
       'Cover_General_ProducedCovers_Total_Counter_Actual_n',
       'Cover_General_RejectedCovers_0_Counter_Actual_n',
       'Upack_Produced_Case_At_Spiral_n'], dtype=object)

#### Mapping to Line Recipe

In [62]:
line_recipe_status_df

,DateTime,Value,line,site
12,2023-04-06 18:07:28.513,21,L313,Amiens
11,2023-04-07 09:00:24.457,51,L313,Amiens
10,2023-04-12 08:26:28.074,57,L313,Amiens
9,2023-04-12 11:19:58.243,42,L313,Amiens
8,2023-04-13 03:20:37.272,51,L313,Amiens
7,2023-04-13 09:32:37.665,52,L313,Amiens
6,2023-04-13 13:08:32.900,22,L313,Amiens
5,2023-04-13 18:29:55.246,21,L313,Amiens
4,2023-04-14 05:33:58.979,37,L313,Amiens
3,2023-04-14 12:47:43.448,25,L313,Amiens


In [63]:
box_totals_df = add_dim_key_time(fact_df=box_totals_df, 
                                 dim_df=line_recipe_status_df.rename(columns={'Value':'recipe_id'}), 
                                 dim_id='recipe_id', fact_dttm='DateTime', dim_dttm='DateTime', 
                                 group_by=['site', 'line'])
print(box_totals_df.shape)
box_totals_df.head()

(108820, 6)


,DateTime,rejects_qty,line,tag,site,recipe_id
53230,2023-04-11 14:39:27.786,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0
53231,2023-04-11 14:39:28.771,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0
53232,2023-04-11 14:39:29.786,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0
53233,2023-04-11 14:39:30.786,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0
53234,2023-04-11 14:46:05.799,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0


#### Mapping to Agile flag

In [65]:
if agile_data_available:
    # add agile flag
    box_totals_df = add_dim_key_time(fact_df=box_totals_df, 
                                      dim_df=agile_status_df.rename(columns={'Value':'agile_flag'}), 
                                      dim_id='agile_flag', fact_dttm='DateTime', dim_dttm='DateTime', 
                                      group_by=['site', 'line'])
    box_totals_df = box_totals_df.assign(agile_flag = box_totals_df['agile_flag'].fillna(0))
else:
    box_totals_df = box_totals_df.assign(agile_flag = 0)

print(box_totals_df.shape)
box_totals_df.head()

(108820, 7)


,DateTime,rejects_qty,line,tag,site,recipe_id,agile_flag
53230,2023-04-11 14:39:27.786,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0
53231,2023-04-11 14:39:28.771,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0
53232,2023-04-11 14:39:29.786,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0
53233,2023-04-11 14:39:30.786,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0
53234,2023-04-11 14:46:05.799,1.0,L313,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0


#### Mapping to Line State

In [66]:
## add line state
box_totals_df = add_dim_key_time(fact_df=box_totals_df.assign(line = box_totals_df['line'].map(line_maping_dict)), 
                                dim_df=line_state_map_df, 
                                dim_id='line_state_id', fact_dttm='DateTime', dim_dttm='DateTime', 
                                group_by=['site', 'line'])
print(box_totals_df.shape)
box_totals_df.head()

(108820, 8)


,DateTime,rejects_qty,line,tag,site,recipe_id,agile_flag,line_state_id
0,2023-04-11 14:39:27.786,1.0,SUAM-31 Packing,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0.0,2100485/15
1,2023-04-11 14:39:28.771,1.0,SUAM-31 Packing,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0.0,2100485/15
2,2023-04-11 14:39:29.786,1.0,SUAM-31 Packing,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0.0,2100485/15
3,2023-04-11 14:39:30.786,1.0,SUAM-31 Packing,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0.0,2100485/15
4,2023-04-11 14:46:05.799,1.0,SUAM-31 Packing,Base_Machine_RejectedBases_Station3_Counter_Ac...,Amiens,51.0,0.0,2100485/17


In [67]:
box_totals_df.columns

Index(['DateTime', 'rejects_qty', 'line', 'tag', 'site', 'recipe_id',
       'agile_flag', 'line_state_id'],
      dtype='object')

#### Aggregating

In [68]:
## aggregating
box_totals_df = box_totals_df.assign(DateTimeMin = box_totals_df['DateTime'].dt.floor(totals_agg_freq))

groupby_cols = ['site', 'line', 'DateTimeMin', 'tag', 
                'agile_flag', 'recipe_id', 'line_state_id']
agg_dict = {'rejects_qty':'sum', 'DateTime':'min'}

box_totals_agg_df = box_totals_df.groupby(groupby_cols).agg(agg_dict).reset_index()
box_totals_agg_df = box_totals_agg_df.loc[box_totals_agg_df['rejects_qty'] != 0]
box_totals_agg_df.head()



,site,line,DateTimeMin,tag,agile_flag,recipe_id,line_state_id,rejects_qty,DateTime
0,Amiens,SUAM-31 Packing,2023-04-11 14:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/15,4.0,2023-04-11 14:39:27.786
1,Amiens,SUAM-31 Packing,2023-04-11 14:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/17,2.0,2023-04-11 14:46:05.799
2,Amiens,SUAM-31 Packing,2023-04-11 14:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/23,3.0,2023-04-11 14:58:33.800
3,Amiens,SUAM-31 Packing,2023-04-11 17:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/57,2.0,2023-04-11 17:50:16.991
4,Amiens,SUAM-31 Packing,2023-04-11 18:00:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/73,1.0,2023-04-11 18:16:54.012


In [69]:
# renaming columns
rename_dict = {'DateTimeMin':'datetime', 'DateTime':'start_time'}

box_totals_agg_df = (box_totals_agg_df
                                    .merge(lines_dim_df[['line_id', 'line']], on = 'line', how='inner')
                                    .merge(sites_dim_df[['site_id', 'site']], on = 'site', how='inner')
                                    .drop(columns=['site', 'line'])
                                    .rename(columns=rename_dict)
                                )
print(box_totals_agg_df.shape)
box_totals_agg_df.head()

(1551, 9)


,datetime,tag,agile_flag,recipe_id,line_state_id,rejects_qty,start_time,line_id,site_id
0,2023-04-11 14:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/15,4.0,2023-04-11 14:39:27.786,10,1
1,2023-04-11 14:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/17,2.0,2023-04-11 14:46:05.799,10,1
2,2023-04-11 14:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/23,3.0,2023-04-11 14:58:33.800,10,1
3,2023-04-11 17:30:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/57,2.0,2023-04-11 17:50:16.991,10,1
4,2023-04-11 18:00:00,Base_Machine_RejectedBases_Station3_Counter_Ac...,0.0,51.0,2100485/73,1.0,2023-04-11 18:16:54.012,10,1


In [70]:
## transposing
box_totals_agg_df = transpose_data_frame(data_frame=box_totals_agg_df,
                                          rows=['site_id', 'line_id', 'datetime', 
                                                'recipe_id', 'agile_flag',
                                                'line_state_id', 'start_time'],
                                          transpose_on='tag')
print(box_totals_agg_df.shape)
box_totals_agg_df.head()

(1305, 17)


,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,start_time,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Base_Machine_ProducedBases_0_Counter_Actual_n,Base_Machine_RejectedBases_Station1_Counter_Actual_n,Base_Machine_RejectedBases_Station2_Counter_Actual_n,Base_Machine_RejectedBases_Station3_Counter_Actual_n,Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n,Cover_General_ExtractedCartons_Total_Counter_Actual_n,Cover_General_ProducedCovers_Total_Counter_Actual_n,Cover_General_RejectedCovers_0_Counter_Actual_n,Upack_Produced_Case_At_Spiral_n
0,1,10,2023-04-11 14:30:00,51.0,0.0,2100485/15,2023-04-11 14:39:27.786,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
1,1,10,2023-04-11 14:30:00,51.0,0.0,2100485/17,2023-04-11 14:46:05.799,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,1,10,2023-04-11 14:30:00,51.0,0.0,2100485/23,2023-04-11 14:58:33.800,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,1,10,2023-04-11 17:30:00,51.0,0.0,2100485/57,2023-04-11 17:50:16.991,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,1,10,2023-04-11 18:00:00,51.0,0.0,2100485/73,2023-04-11 18:16:54.012,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [71]:
box_totals_agg_df.columns

Index(['site_id', 'line_id', 'datetime', 'recipe_id', 'agile_flag',
       'line_state_id', 'start_time',
       'Base_Machine_ExtractedCartons_0_Counter_Actual_n',
       'Base_Machine_ProducedBases_0_Counter_Actual_n',
       'Base_Machine_RejectedBases_Station1_Counter_Actual_n',
       'Base_Machine_RejectedBases_Station2_Counter_Actual_n',
       'Base_Machine_RejectedBases_Station3_Counter_Actual_n',
       'Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n',
       'Cover_General_ExtractedCartons_Total_Counter_Actual_n',
       'Cover_General_ProducedCovers_Total_Counter_Actual_n',
       'Cover_General_RejectedCovers_0_Counter_Actual_n',
       'Upack_Produced_Case_At_Spiral_n'],
      dtype='object')

In [72]:
## aggregating

groupby_cols = ['site_id', 'line_id', 'datetime', 
                'recipe_id', 'agile_flag','line_state_id']
agg_dict = {'Base_Machine_ExtractedCartons_0_Counter_Actual_n':'sum', 
            'Base_Machine_ProducedBases_0_Counter_Actual_n':'sum',
            'Cover_General_ExtractedCartons_Total_Counter_Actual_n':'sum',
            'Cover_General_ProducedCovers_Total_Counter_Actual_n':'sum',
            'Base_Machine_RejectedBases_Station1_Counter_Actual_n':'sum',
            'Base_Machine_RejectedBases_Station2_Counter_Actual_n':'sum',
            'Base_Machine_RejectedBases_Station3_Counter_Actual_n':'sum',
            'Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n':'sum',
            'Cover_General_RejectedCovers_0_Counter_Actual_n':'sum',
            'Upack_Produced_Case_At_Spiral_n':'sum',
            'start_time':'min'}

box_totals_agg_df = box_totals_agg_df.groupby(groupby_cols).agg(agg_dict).reset_index()
print(box_totals_agg_df.shape)
box_totals_agg_df.head()


(447, 17)


,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Base_Machine_ProducedBases_0_Counter_Actual_n,Cover_General_ExtractedCartons_Total_Counter_Actual_n,Cover_General_ProducedCovers_Total_Counter_Actual_n,Base_Machine_RejectedBases_Station1_Counter_Actual_n,Base_Machine_RejectedBases_Station2_Counter_Actual_n,Base_Machine_RejectedBases_Station3_Counter_Actual_n,Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n,Cover_General_RejectedCovers_0_Counter_Actual_n,Upack_Produced_Case_At_Spiral_n,start_time
0,1,10,2023-04-11 14:30:00,51.0,0.0,2100485/15,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2023-04-11 14:39:27.786
1,1,10,2023-04-11 14:30:00,51.0,0.0,2100485/17,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2023-04-11 14:46:05.799
2,1,10,2023-04-11 14:30:00,51.0,0.0,2100485/23,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2023-04-11 14:58:33.800
3,1,10,2023-04-11 17:30:00,51.0,0.0,2100485/57,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2023-04-11 17:50:16.991
4,1,10,2023-04-11 18:00:00,51.0,0.0,2100485/73,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2023-04-11 18:16:54.012


In [73]:
# if site_cd == 'ami':
#             agg_dict['Upack_Produced_Case_At_Spiral_n'] = 'sum'
# elif site_cd =='url':
#     agg_dict['UPack_For_Proficy_Produced_Cases'] = 'sum'
# else:
#     pass



In [74]:

# box_totals_agg_df = box_totals_agg_df.groupby(groupby_cols).agg(agg_dict).reset_index()
# print(f'Shape after aggregating box totals data: {box_totals_agg_df.shape}') 

In [75]:

# box_totals_agg_df = box_totals_agg_df.groupby(groupby_cols).agg(agg_dict).reset_index()
# print(f'Shape after aggregating box totals data: {box_totals_agg_df.shape}')   

if site_cd == 'ami':
    box_totals_agg_df = box_totals_agg_df.rename(columns = {'Upack_Produced_Case_At_Spiral_n':'UPack_For_Proficy_Produced_Cases'})
    ordered_columns = [c for c in box_totals_agg_df.columns if c!='start_time'] + ['start_time']
    box_totals_agg_df = box_totals_agg_df[ordered_columns]


In [76]:
#box_totals_agg_df_L34 = box_totals_agg_df[ordered_columns]
#box_totals_agg_df_L34

In [77]:
#box_totals_agg_df_L27_second_chunk = box_totals_agg_df[ordered_columns]
#box_totals_agg_df_L27_second_chunk

In [78]:
#box_totals_agg_df_L27_second_chunk.to_excel('..\Data\incremental_loading\second_chunk.xlsx', index=False)

In [79]:
# box_totals_agg_df_L27_first_chunk = box_totals_agg_df[ordered_columns]
# box_totals_agg_df_L27_first_chunk

In [80]:
# box_totals_agg_df_L27_first_chunk

In [81]:
#box_totals_agg_df_L27_first_chunk.to_csv('..\Data\incremental_loading\Firstchunk.csv')

In [82]:
#box_totals_agg_df_L27_first_chunk.to_excel('..\Data\incremental_loading\First_chunk.xlsx', index=False)

In [83]:
#box_totals_agg_dfn_ = pd.concat([box_totals_agg_df_L27_first_chunk,box_totals_agg_df_L27_second_chunk], ignore_index = True)
#box_totals_agg_dfn_

In [84]:
#box_totals_agg_dfn_.shape

In [85]:
#box_totals_agg_dfn_ = box_totals_agg_dfn_.drop_duplicates(subset=['site_id', 'line_id', 'datetime', 'recipe_id', 'line_state_id'], keep = 'last')



In [86]:
#box_totals_agg_dfn_.shape

In [87]:
#loading

In [88]:
truncate_date = start_time + timedelta(days=site_history_buffer_days)
print(truncate_date)

2023-04-10 05:30:00


In [69]:
#truncate_date = start_time
#print(truncate_date)

2022-12-09 13:00:00


In [70]:
site_sud_id = sites_dim_df.loc[sites_dim_df['site'] == site_name, 'site_id'].iloc[0]
box_totals_agg_df = box_totals_agg_df.loc[box_totals_agg_df['datetime'] >= truncate_date]
print(f'Truncating after {truncate_date}. Shape of truncated box rejects data: {box_totals_agg_df.shape}')


Truncating after 2022-12-09 13:00:00. Shape of truncated box rejects data: (4929, 17)


In [71]:
truncate_query = queries.TRUNCATE_REJECTS_TABLE.format(box_production_table_nm, site_sud_id, truncate_date)
print(truncate_query)

 
    delete from SUD_BOX_TOTALS
    where site_id = 1 and datetime >= '2022-12-09 13:00:00'



In [72]:
execute_db_query(conn_string=datalab_db_conn, query=truncate_query, access_token=datalab_db_access_token)
insert_df_to_mssql_db(df=box_totals_agg_df,
                    db_conn=datalab_db_conn,
                    db_table=box_production_table_nm,
                    access_token=datalab_db_access_token)


In [ ]:
#box_totals_agg_dfn_.shape

# Loading

### Updating SUD_BOX_REJECTS table

In [113]:
truncate_date = start_time + timedelta(days=site_history_buffer_days)
print(truncate_date)

2023-01-01 06:00:00


In [104]:
#truncate_date = start_time + timedelta(days=site_history_buffer_days)
truncate_date = start_time
print(truncate_date)
site_sud_id = sites_dim_df.loc[sites_dim_df['site'] == site_name, 'site_id'].iloc[0]
box_rejects_agg_df = box_rejects_agg_df.loc[box_rejects_agg_df['datetime'] >= truncate_date]
           

2023-02-01 06:00:00


In [105]:
truncate_query = queries.TRUNCATE_REJECTS_TABLE.format(box_rejects_table_nm, site_sud_id, truncate_date)
print(truncate_query)


 
    delete from SUD_BOX_REJECTS
    where site_id = 1 and datetime >= '2023-02-01 06:00:00'



In [107]:
execute_db_query(conn_string=datalab_db_conn, query=truncate_query, access_token=datalab_db_access_token)
insert_df_to_mssql_db(df=box_rejects_agg_df,
                      db_conn=datalab_db_conn,
                      db_table=box_rejects_table_nm,
                      access_token=datalab_db_access_token)

In [46]:
insert_df_to_mssql_db(df=box_rejects_agg_df_,
                      db_conn=datalab_db_conn,
                      db_table=box_rejects_table_nm,
                      access_token=datalab_db_access_token)

#### Updating ETL_RUN_SUMMARY table

In [83]:

# updating etl status table
job_step_start_dttm = datetime.now()
extract_overview_df = (box_rejects_agg_df.groupby(['site_id', 'line_id'])
                                        .agg({'datetime':'max', 'line_state_id':'count'})
                                        .reset_index()
                                        .rename(columns={'line_state_id':'n_records', 'datetime':'start_time'}))
new_last_available_dttm = extract_overview_df['start_time'].min()

job_summary_record = get_run_summary_entry(job_id, job_nm, box_rejects_table_nm,
                                            job_step_start_dttm, datetime.now(),
                                            extract_overview_df['n_records'].sum(),
                                            np.nan, 'start_time', 
                                            new_last_available_dttm, np.nan)


insert_df_to_mssql_db(df=job_summary_record,
                          db_conn=datalab_db_conn,
                          db_table=job_summary_table_nm,
                          access_token=datalab_db_access_token)

### Updating SUD_BOX_TOTALS table

In [84]:
truncate_date = start_time + timedelta(days=site_history_buffer_days)
site_sud_id = sites_dim_df.loc[sites_dim_df['site'] == site_name, 'site_id'].iloc[0]
box_totals_agg_df = box_totals_agg_df.loc[box_totals_agg_df['datetime'] >= truncate_date]

In [85]:
truncate_query = queries.TRUNCATE_REJECTS_TABLE.format(box_production_table_nm, site_sud_id, truncate_date)
print(truncate_query)


 
    delete from SUD_BOX_TOTALS
    where site_id = 1 and datetime >= '2022-08-16 12:00:00'



In [86]:
truncate_query = queries.TRUNCATE_REJECTS_TABLE.format(box_production_table_nm, site_sud_id, truncate_date)
execute_db_query(conn_string=datalab_db_conn, query=truncate_query, access_token=datalab_db_access_token)
insert_df_to_mssql_db(df=box_totals_agg_df,
                    db_conn=datalab_db_conn,
                    db_table=box_production_table_nm,
                    access_token=datalab_db_access_token)


### Updating SUD_ETL_RUN_SUMMARY table

In [87]:

# updating etl status table
job_step_start_dttm = datetime.now()
extract_overview_df = (box_totals_agg_df.groupby(['site_id', 'line_id'])
                                        .agg({'datetime':'max', 'line_state_id':'count'})
                                        .reset_index()
                                        .rename(columns={'line_state_id':'n_records', 'datetime':'start_time'}))
new_last_available_dttm = extract_overview_df['start_time'].min()

job_summary_record = get_run_summary_entry(job_id, job_nm, box_production_table_nm,
                                                            job_step_start_dttm, datetime.now(),
                                                            extract_overview_df['n_records'].sum(),
                                                            np.nan, 'start_time', 
                                                            new_last_available_dttm, np.nan)


insert_df_to_mssql_db(df=job_summary_record,
                          db_conn=datalab_db_conn,
                          db_table=job_summary_table_nm,
                          access_token=datalab_db_access_token)

# checking data

In [47]:
check_query = """
            select * from SUD_BOX_TOTALS where datetime  >= '2023-01-01'
"""
df = pd.read_sql(sql=check_query, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD sites dimention table loaded, shape: {df.shape}')
df

SUD sites dimention table loaded, shape: (50095, 17)


,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,Base_Machine_ExtractedCartons_0_Counter_Actual_n,Base_Machine_ProducedBases_0_Counter_Actual_n,Cover_General_ExtractedCartons_Total_Counter_Actual_n,Cover_General_ProducedCovers_Total_Counter_Actual_n,Base_Machine_RejectedBases_Station1_Counter_Actual_n,Base_Machine_RejectedBases_Station2_Counter_Actual_n,Base_Machine_RejectedBases_Station3_Counter_Actual_n,Base_Machine_RejectedBases_Total_NOT_Extracted_Betw_Holders_n,Cover_General_RejectedCovers_0_Counter_Actual_n,UPack_For_Proficy_Produced_Cases,start_time
0,1,73,2023-01-05 16:00:00,42,0,F1F0C4D4-5E0A-43CD-B925-25C4CB87F9A9/7,0,0,21,7,0,0,0,0,0,23,2023-01-05 16:16:59.707
1,1,73,2023-02-20 08:30:00,16,0,84F72D37-3D02-488B-BCE6-5F0DD856CDC9/1,105,95,75,66,18,1,0,0,1,0,2023-02-20 08:59:07.477
2,1,73,2023-01-05 16:00:00,42,0,F1F0C4D4-5E0A-43CD-B925-25C4CB87F9A9/8,166,160,181,177,0,0,0,0,4,13,2023-01-05 16:17:56.667
3,1,73,2023-01-05 16:00:00,42,0,F1F0C4D4-5E0A-43CD-B925-25C4CB87F9A9/9,217,215,220,219,0,0,0,0,2,55,2023-01-05 16:19:50.663
4,1,73,2023-01-05 16:30:00,42,0,F1F0C4D4-5E0A-43CD-B925-25C4CB87F9A9/12,2104,2117,2097,2094,0,1,0,0,20,528,2023-01-05 16:30:00.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50090,1,73,2023-02-20 08:00:00,42,0,7E37AD3A-31C6-4EF5-920D-17858791E039/0,63,70,62,62,0,0,0,0,0,18,2023-02-20 08:12:08.393
50091,1,73,2023-02-20 08:00:00,42,0,83C74D1C-D4DB-49EC-B777-B3A48B9F37C7/0,117,117,117,120,0,0,0,0,0,29,2023-02-20 08:28:55.310
50092,1,73,2023-02-20 08:30:00,16,0,4DA9F394-BDAA-45DF-9B32-3F9C08CE09CE/0,12,0,0,0,0,0,0,0,0,0,2023-02-20 08:57:03.480
50093,1,73,2023-02-20 08:30:00,16,0,4DA9F394-BDAA-45DF-9B32-3F9C08CE09CE/1,2,0,0,0,0,0,0,0,0,0,2023-02-20 08:57:08.467


In [48]:
df[['site_id', 'line_id','line_state_id','recipe_id','datetime']].duplicated().any()

False

In [49]:
df[['recipe_id','line_state_id','datetime']].duplicated().any()

False

# manual reject

In [166]:
manual_reject_query = """

select bt.[site_id]
      ,bt.[line_id]
      ,bt.[datetime]
      ,bt.[recipe_id]
      ,bt.[agile_flag]
      ,bt.[line_state_id]
      ,bt.[Cover_General_ExtractedCartons_Total_Counter_Actual_n] as n_produced_covers
      ,bt.[Base_Machine_ProducedBases_0_Counter_Actual_n] as n_packaged_packs
      ,COALESCE(br.rejects_qty, 0) as n_rejects
      ,bt.[Cover_General_ExtractedCartons_Total_Counter_Actual_n] - bt.[Base_Machine_ProducedBases_0_Counter_Actual_n] - COALESCE(br.rejects_qty, 0) as rejects_qty
      ,104 as [reject_cause_id]
  from [dbo].[SUD_BOX_TOTALS] as bt
  left join (select                r.[site_id]
                ,r.[line_id]
                ,r.[datetime]
                ,r.[recipe_id]
                ,r.[agile_flag]
                ,r.[line_state_id]
                ,sum(r.[rejects_qty]) as rejects_qty
             from [dbo].[SUD_BOX_REJECTS] as r
             inner join [dbo].[SUD_BOX_CAUSE_REJECTS_DIM] as cr 
                on r.[reject_cause_id] = cr.[reject_cause_id]
             where (cr.[type] = 'Cover')                or (cr.[type] = 'Cover+Base' and cr.[machine] = 'Mettler')                or (cr.[type] = 'Cover+Base' and cr.[machine] = 'Base machine')             group by [site_id]
                ,[line_id]
                ,[datetime]
                ,[recipe_id]
                ,[agile_flag]
                ,[line_state_id]
            ) as br
on bt.[site_id] = br.[site_id] and   bt.[line_id] = br.[line_id] and   bt.[datetime] = br.[datetime] and   bt.[recipe_id] = br.[recipe_id] and   bt.[agile_flag] = br.[agile_flag] and   bt.[line_state_id] = br.[line_state_id]


"""

In [167]:
df = pd.read_sql(sql=manual_reject_query, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD sites dimention table loaded, shape: {df.shape}')

SUD sites dimention table loaded, shape: (155617, 11)


In [168]:
df

,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,n_produced_covers,n_packaged_packs,n_rejects,rejects_qty,reject_cause_id
0,1,20,2022-10-04 14:30:00,21,0,1462084/5,1589,1578,0,11,104
1,1,20,2022-10-04 14:30:00,21,0,1462084/6,0,2,0,-2,104
2,1,20,2022-10-04 15:30:00,21,0,1462084/11,689,669,0,20,104
3,1,20,2022-10-04 16:00:00,21,0,1462084/12,0,0,0,0,104
4,1,20,2022-10-04 16:30:00,21,0,1462084/20,28,7,0,21,104
...,...,...,...,...,...,...,...,...,...,...,...
155612,5,70,2023-03-10 12:00:00,41,0,253328/23,1643,1609,25,9,104
155613,5,70,2023-03-10 12:30:00,41,0,253328/33,1807,1824,27,-44,104
155614,5,70,2023-03-10 12:30:00,41,0,253328/37,338,321,22,-5,104
155615,5,70,2023-03-10 12:30:00,41,0,253328/41,64,26,3,35,104


In [169]:
df_ = df.loc[(df['rejects_qty'] < 0)]
df_.line_id.unique()

array([20, 23, 73, 70], dtype=int64)

In [170]:
df.loc[(df['rejects_qty'] < 0) & (df['datetime'] >= '2023-03-10') & (df['line_id'] == 23)].head(20).sort_values(by = ['datetime'])

,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,n_produced_covers,n_packaged_packs,n_rejects,rejects_qty,reject_cause_id
58303,1,23,2023-03-10 00:30:00,11,0,2095988/25,4140,4130,18,-8,104
58306,1,23,2023-03-10 02:30:00,11,0,2095988/37,3663,3635,38,-10,104
57230,1,23,2023-03-10 03:00:00,11,0,2095988/37,2971,2942,53,-24,104
58309,1,23,2023-03-10 05:00:00,11,0,2096025/15,3098,3098,6,-6,104
58310,1,23,2023-03-10 05:30:00,11,0,2096025/15,192,203,10,-21,104
57585,1,23,2023-03-10 06:00:00,41,0,2096055/13,17,17,1,-1,104
57586,1,23,2023-03-10 06:00:00,41,0,2096055/9,13,14,0,-1,104
55459,1,23,2023-03-10 06:30:00,41,0,2096055/40,40,40,2,-2,104
57587,1,23,2023-03-10 06:30:00,41,0,2096055/22,111,130,30,-49,104
57589,1,23,2023-03-10 07:00:00,41,0,2096055/50,410,400,29,-19,104


### for base

In [176]:
query_base ="""

select bt.[site_id]
      ,bt.[line_id]
      ,bt.[datetime]
      ,bt.[recipe_id]
      ,bt.[agile_flag]
      ,bt.[line_state_id]
      ,bt.[Cover_General_ExtractedCartons_Total_Counter_Actual_n] as n_produced_covers
      ,bt.[Base_Machine_ProducedBases_0_Counter_Actual_n] as n_packaged_packs
      ,COALESCE(br.rejects_qty, 0) as n_rejects
      ,bt.[Cover_General_ExtractedCartons_Total_Counter_Actual_n] - bt.[Base_Machine_ProducedBases_0_Counter_Actual_n] - COALESCE(br.rejects_qty, 0) as rejects_qty
      ,998 as [reject_cause_id]
  from [dbo].[SUD_BOX_TOTALS] as bt
  left join (select                r.[site_id]
                ,r.[line_id]
                ,r.[datetime]
                ,r.[recipe_id]
                ,r.[agile_flag]
                ,r.[line_state_id]
                ,sum(r.[rejects_qty]) as rejects_qty
             from [dbo].[SUD_BOX_REJECTS] as r
             inner join [dbo].[SUD_BOX_CAUSE_REJECTS_DIM] as cr 
                on r.[reject_cause_id] = cr.[reject_cause_id]
             where (cr.[type] = 'Cover')                or (cr.[type] = 'Cover+Base' and cr.[machine] = 'Mettler')                or (cr.[type] = 'Cover+Base' and cr.[machine] = 'Base machine')             group by [site_id]
                ,[line_id]
                ,[datetime]
                ,[recipe_id]
                ,[agile_flag]
                ,[line_state_id]
            ) as br
on bt.[site_id] = br.[site_id] and   bt.[line_id] = br.[line_id] and   bt.[datetime] = br.[datetime] and   bt.[recipe_id] = br.[recipe_id] and   bt.[agile_flag] = br.[agile_flag] and   bt.[line_state_id] = br.[line_state_id]
"""

In [177]:
df_base = pd.read_sql(sql=query_base, con=get_db_connection(datalab_db_conn, datalab_db_access_token))
print(f'SUD sites dimention table loaded, shape: {df_base.shape}')

SUD sites dimention table loaded, shape: (155939, 11)


In [178]:
df_base

,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,n_produced_covers,n_packaged_packs,n_rejects,rejects_qty,reject_cause_id
0,1,20,2022-10-04 14:00:00,21,0,1462084/4,11,33,0,-22,998
1,1,20,2022-10-04 17:00:00,21,0,1462084/24,0,0,0,0,998
2,1,20,2022-10-04 17:30:00,21,0,1462084/33,136,127,0,9,998
3,1,20,2022-10-04 17:30:00,21,0,1462084/37,225,210,0,15,998
4,1,20,2022-10-04 17:30:00,21,0,1462084/41,731,683,0,48,998
...,...,...,...,...,...,...,...,...,...,...,...
155934,5,70,2023-03-10 15:30:00,41,0,253369/4,246,250,4,-8,998
155935,5,70,2023-03-10 16:30:00,51,0,253378/11,16,15,0,1,998
155936,5,70,2023-03-10 18:00:00,51,0,253378/16,647,648,6,-7,998
155937,5,70,2023-03-10 19:30:00,51,0,253378/18,3090,3029,71,-10,998


In [179]:
df_base.loc[df_base.rejects_qty < 0]

,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,n_produced_covers,n_packaged_packs,n_rejects,rejects_qty,reject_cause_id
0,1,20,2022-10-04 14:00:00,21,0,1462084/4,11,33,0,-22,998
10,1,20,2022-10-04 22:30:00,37,0,1462102/1,11,21,0,-10,998
11,1,20,2022-10-04 23:00:00,37,0,1462102/1,10,18,0,-8,998
12,1,20,2022-10-05 00:00:00,37,0,1462102/9,130,136,0,-6,998
15,1,20,2022-10-05 02:00:00,37,0,1462102/29,271,283,0,-12,998
...,...,...,...,...,...,...,...,...,...,...,...
155925,5,70,2023-03-09 17:00:00,41,0,253209/34,369,378,5,-14,998
155929,5,70,2023-03-10 10:30:00,41,0,253328/1,36,35,27,-26,998
155934,5,70,2023-03-10 15:30:00,41,0,253369/4,246,250,4,-8,998
155936,5,70,2023-03-10 18:00:00,51,0,253378/16,647,648,6,-7,998


In [180]:
df_base.loc[(df_base['rejects_qty'] < 0) & (df_base['datetime'] >= '2023-03-10') & (df_base['line_id'] == 23)].head(20).sort_values(by = ['datetime'])

,site_id,line_id,datetime,recipe_id,agile_flag,line_state_id,n_produced_covers,n_packaged_packs,n_rejects,rejects_qty,reject_cause_id
49842,1,23,2023-03-10 00:30:00,11,0,2095988/25,4140,4130,18,-8,998
46270,1,23,2023-03-10 02:00:00,11,0,2095988/33,3841,3857,33,-49,998
49845,1,23,2023-03-10 02:30:00,11,0,2095988/37,3663,3635,38,-10,998
49848,1,23,2023-03-10 05:00:00,11,0,2096025/15,3098,3098,6,-6,998
49849,1,23,2023-03-10 05:30:00,11,0,2096025/15,192,203,10,-21,998
46275,1,23,2023-03-10 07:00:00,41,0,2096055/40,283,225,59,-1,998
46278,1,23,2023-03-10 07:30:00,41,0,2096055/58,222,234,35,-47,998
49853,1,23,2023-03-10 07:30:00,41,0,2096055/52,20,16,8,-4,998
46279,1,23,2023-03-10 08:00:00,41,0,2096055/63,5,19,0,-14,998
49855,1,23,2023-03-10 08:30:00,41,0,2096055/68,1502,1400,132,-30,998


# Adding tags to SUD_PBI_TOOL_TAGS table

### function for generating dataframe for tags info

In [70]:
# def taglist_table(site_nm, histo_topic, tool_nm):
#     """ Function to generate a tag list table to be populated into SUD_PBI_TOOL_TAG in Datalab DB
    
#     Args:
#         site_nm (str): Site name
#         histo_topic (str): Site specific historian topic
#         tool_nm (str): names of the dashboard/tool

#     Returns:
#         pd.DataFrame: dataframe containing tag used in the specific tool in format matching the SUD_PBI_TOOL_TAG table 
    
#     """
#     tag_df=pd.DataFrame({'TagName':reject_cause_tags})
#     tag_df=tag_df.assign(ToolName=tool_nm,
#                         Site=site_nm,
#                         Line=(tag_df['TagName'].str.split('.', n=1, expand=True)[1]
#                         .str.split('_',n=1,expand=True)[0]),
#                         LastModifiedDate=datetime.now(),
#                         Technology='BOX'
                    
#                         )
#     tag_df=tag_df[['ToolName','Site','Line','TagName','LastModifiedDate','Technology']]
#     return tag_df

In [71]:
# tags_df = taglist_table(site_name, site_servers['historian']['proficy']['topic'], pbi_refresh_app)
# tags_df.head()

### Truncating and load data into SUD_PBI_TOOL_TAGS table

In [72]:
# truncate_query_tmp=""" delete from {}
#                        where Site = '{}' and ToolName='{}'
#                     """

# truncate_query=truncate_query_tmp.format(tags_df, site_name, pbi_refresh_app)

In [73]:
# def truncate_and_load(conn_string, truncate_query, access_token,df, table_db):
#     execute_db_query(conn_string,truncate_query,access_token )
#     insert_df_to_mssql_db(df=df, 
#                        db_conn=datalab_db_conn,
#                        db_table=table_db, 
#                        access_token=access_token,
#                         if_exists='append'
#                         )

In [76]:
#truncate_and_load(datalab_db_conn, truncate_query, datalab_db_access_token, tags_df)